In [2]:
# Import libraries
import json
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer

In [3]:
# Initialize model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [12]:

# Load your pre-computed data
with open('../data/EmbeddingFiles/canonical_names.json', 'r') as f:
    canonical_exercises = json.load(f)

# Load FAISS index
exercise_index = faiss.read_index('../data/EmbeddingFiles/exercise_index.faiss')

In [13]:
def extract_exercise_name_debug(user_query, top_k=5):
    """Debug version to see what's happening"""
    # Embed the user query
    query_embedding = model.encode(user_query)
    query_embedding = np.array([query_embedding]).astype('float32')
    
    # Search
    distances, indices = exercise_index.search(query_embedding, top_k)
    
    print(f"\nQuery: '{user_query}'")
    print(f"Top {top_k} matches:")
    for i in range(top_k):
        idx = indices[0][i]
        dist = distances[0][i]
        exercise = canonical_exercises[idx]
        print(f"  {i+1}. {exercise['canonical_name']} (distance: {dist:.4f})")
    
    # Return the best match
    best_idx = indices[0][0]
    return canonical_exercises[best_idx]['canonical_name']

def extract_exercise_name(user_query, threshold=10.0):
    """Extract exercise name using FAISS"""
    query_embedding = model.encode(user_query)
    query_embedding = np.array([query_embedding]).astype('float32')
    
    distances, indices = exercise_index.search(query_embedding, 1)
    
    best_idx = indices[0][0]
    best_distance = distances[0][0]
    
    if best_distance < threshold:
        return canonical_exercises[best_idx]['canonical_name']
    
    return None

In [14]:
# Test
test_queries = [
    "How's my squat doing?",
    "Am I improving on bench?",
    "Show deadlift progress",
    "Track overhead press gains"
]

for query in test_queries:
    exercise = extract_exercise_name(query)
    print(f"Query: '{query}' -> Exercise: '{exercise}'")

Query: 'How's my squat doing?' -> Exercise: 'Bodyweight Squat'
Query: 'Am I improving on bench?' -> Exercise: 'Bench Jump'
Query: 'Show deadlift progress' -> Exercise: 'Band Deadlift'
Query: 'Track overhead press gains' -> Exercise: 'Dumbbell Tricep Press (Overhead)'


#### Will have to add a weight to exercises because deadlift != Band Deadlift but ideally barbell deadlift